In [2]:
import sys
sys.path.append('./hw4/python')
import numpy as np

import needle as ndl
import needle.backend_ndarray.ndarray as nd


In [6]:
device = ndl.cpu()
np.random.seed(0)
dataset = ndl.data.CIFAR10Dataset("/home/justin/Needle/hw4/data/cifar-10-batches-py", train=True)
dataloader = ndl.data.DataLoader(\
            dataset=dataset,
            batch_size=128,
            shuffle=False
            # collate_fn=ndl.data.collate_ndarray,
            # drop_last=False,
            # device=device,
            # dtype="float32"
            )
from hw4.apps.models import ResNet9
np.random.seed(0)
model = ResNet9(device=device, dtype="float32")



In [7]:

def one_iter_of_cifar10_training(dataloader, model, niter=1, loss_fn=ndl.nn.SoftmaxLoss(), opt=None, device=None):
    np.random.seed(4)
    model.train()
    correct, total_loss = 0, 0
    i = 1
    for batch in dataloader:
        opt.reset_grad()
        X, y = batch
        X,y = ndl.Tensor(X, device=device), ndl.Tensor(y, device=device)
        out = model(X)
        correct += np.sum(np.argmax(out.numpy(), axis=1) == y.numpy())
        loss = loss_fn(out, y)
        total_loss += loss.data.numpy() * y.shape[0]
        loss.backward()
        opt.step()
        if i >= niter:
            break
        i += 1
    return correct/(y.shape[0]*niter), total_loss/(y.shape[0]*niter)

In [9]:

out = one_iter_of_cifar10_training(dataloader, model, opt=ndl.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001), device=device)
assert np.linalg.norm(np.array(list(out), dtype=object) - np.array([0.09375, 3.5892258])) < 1e-2

AttributeError: 'CPUDevice' object has no attribute 'Array'

In [ ]:
a = ndl.Tensor(np.arange(10).reshape((2,5)))
a

In [ ]:
b = a.reshape((5,2))
b

In [ ]:
c = b.reshape((2,5))
c.backward()
a.grad